# Hurto Automotores, Comercio & Financieras

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [78]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'
CODIGO_DANE_PATH = '/content/drive/MyDrive/Police-Data/dane_codes.xlsx'

## Libraries

In [30]:
import pandas as pd
import numpy as np
import os
import glob

In [31]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

## Read Files

In [32]:
folders = ["Hurto_Automotores","Hurto_Comercio","Hurto_Financieras"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [33]:
folder_read(folders)

## Hurto Automotores

In [107]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [108]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Hurto_Automotores']]
hurto_automotores = pd.concat(standardized_dataframes, ignore_index=True)

In [109]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['LLAVE MAESTRA','ARMA DE FUEGO','CONTUNDENTES','ESCOPOLAMINA','ARMA BLANCA / CORTOPUNZANTE','PUNZANTES',
                             'CORTOPUNZANTES','CORTANTES','SIN EMPLEO DE ARMAS','JERINGA']
hurto_automotores["AGRUPA_EDAD_PERSONA"] = hurto_automotores["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_automotores["GENERO"] = hurto_automotores["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_automotores["ARMAS_MEDIOS"] = hurto_automotores["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [110]:
dane_codes = pd.read_excel(CODIGO_DANE_PATH,sheet_name='Municipios_dane')
def clean_data(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str).str[:5]
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  filtered_df.loc[:, 'CODIGO_DANE'] = filtered_df['CODIGO_DANE'].astype(int)
  # Adjust upper case
  filtered_df = filtered_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)
  # fill NaN with most common value
  filtered_df['MUNICIPIO'] = filtered_df.groupby('CODIGO_DANE')['MUNICIPIO'].transform(lambda x: x.ffill().bfill())
  # Merge DANE CODE
  merged_df = filtered_df.merge(dane_codes, on='CODIGO_DANE', how='left', suffixes=('', '_csv'))
  # fill NaNs
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO'].fillna(merged_df['MUNICIPIO_csv'])
  merged_df = merged_df.drop(columns=['MUNICIPIO_csv'])
  merged_df['CAPITAL'] = merged_df['MUNICIPIO'].apply(lambda municipio: 1 if municipio[-3:] == 'CT)' else 0)
  return merged_df

In [111]:
hurto_filtered = clean_data(hurto_automotores)

In [112]:
hurto_automotores_output = hurto_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [113]:
hurto_automotores_output.to_csv(OUTPUT_PATH_FILE+"/hurto_automotores.csv",index=False)

## Hurto Comercio

In [114]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [115]:
dataframes_by_folder['Hurto_Comercio'][0]['ARMA_medio_temp'] = dataframes_by_folder['Hurto_Comercio'][0]['CODIGO DANE']
dataframes_by_folder['Hurto_Comercio'][0]['CODIGO DANE'] = dataframes_by_folder['Hurto_Comercio'][0]['ARMAS MEDIOS']
dataframes_by_folder['Hurto_Comercio'][0]['ARMAS MEDIOS'] = dataframes_by_folder['Hurto_Comercio'][0]['ARMA_medio_temp']
dataframes_by_folder['Hurto_Comercio'][0].drop(columns='ARMA_medio_temp',inplace=True)

In [116]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Hurto_Comercio']]
hurto_comercio = pd.concat(standardized_dataframes, ignore_index=True)

In [117]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','CORTANTES','ESCOPOLAMINA','PUNZANTES','SIN EMPLEO DE ARMAS',
                             'JERINGA','LLAVE MAESTRA','PALANCAS']
hurto_comercio["AGRUPA_EDAD_PERSONA"] = hurto_comercio["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_comercio["GENERO"] = hurto_comercio["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_comercio["ARMAS_MEDIOS"] = hurto_comercio["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [118]:
hurto_comercio_filtered = clean_data(hurto_comercio)

In [119]:
hurto_comercio_output = hurto_comercio_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [120]:
hurto_comercio_output.to_csv(OUTPUT_PATH_FILE+"/hurto_comercio.csv",index=False)

## Hurto Financieras

In [121]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [122]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Hurto_Financieras']]
hurto_financieras = pd.concat(standardized_dataframes, ignore_index=True)

In [123]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA DE FUEGO','SIN EMPLEO DE ARMAS','CONTUNDENTES','CORTANTES','ARMA BLANCA / CORTOPUNZANTE',
                             'ARTEFACTO EXPLOSIVO/CARGA DINAMITA','PUNZANTES','LLAVE MAESTRA','PALANCAS']
hurto_financieras["AGRUPA_EDAD_PERSONA"] = hurto_financieras["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
hurto_financieras["GENERO"] = hurto_financieras["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
hurto_financieras["ARMAS_MEDIOS"] = hurto_financieras["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [124]:
hurto_financieras_filtered = clean_data(hurto_financieras)

In [125]:
hurto_financieras_output = hurto_financieras_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [126]:
hurto_financieras_output.to_csv(OUTPUT_PATH_FILE+"/hurto_financieras.csv",index=False)